In [1]:
import numpy as np 
import urdf2casadi.urdfparser as u2c
from urdf2casadi.geometry import plucker
from urdf_parser_py.urdf import URDF, Pose
import PyKDL as kdl
import kdl_parser_py.urdf as kdlurdf
from timeit import Timer, timeit, repeat
import rbdl
import pybullet as pb


In [2]:
def median(lst):
    n = len(lst)
    if n < 1:
            return None
    if n % 2 == 1:
            return sorted(lst)[n//2]
    else:
            return sum(sorted(lst)[n//2-1:n//2+1])/2.0
        
def average(lst): 
    return sum(lst) / len(lst) 


def c_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToCoriolis(q_kdl, qdot_kdl, g_kdl)


def c_u2c_func():   
    for j in range(njoints):
        q_none = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    c_u2c(q_none, qdot_none)
    
def c_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    
    c_pb = pb.calculateInverseDynamics(pbmodel, q_none, qdot_none, qddot)
    
def c_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.NonlinearEffects(rbdlmodel, q_np, qdot_np, c_rbdl)


def g_kdl_func():
    for j in range(njoints):
        q_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        #qdot_kdl[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    kdl.ChainDynParam(kdlmodel, gravity_kdl).JntToGravity(q_kdl, g_kdl)


def g_u2c_func():   
    for j in range(njoints):
        q_none = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        #qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    g_u2c(q_none)
    
def g_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        #qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    
    g_pb = pb.calculateInverseDynamics(pbmodel, q_none, qdot, qddot)
    
def g_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        #qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.InverseDynamics(rbdlmodel, q_np, qdot_np, qddot_np, g_rbdl)
    
    
def ID_u2c_func():   
    for j in range(njoints):
        q_none = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    ID_u2c(q_none, qdot_none, qddot_none)
    
def ID_pb_func():
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
    
    ID_pb = pb.calculateInverseDynamics(pbmodel, q_none, qdot_none, qddot_none)
    
def ID_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        qddot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.InverseDynamics(rbdlmodel, q_np, qdot_np, qddot_np, ID_rbdl)



In [3]:
#pbmodel = pb.loadURDF("pantilt.urdf")
sim = pb.connect(pb.DIRECT)
#pbmodel = pb.loadURDF("/urdf4timing/1dof.urdf")
ndofs = 10
urdf_nr = list(range(ndofs + 1))
urdf_nr.pop(0)


nitr = 1000
nrepeat = 100
    

path_to_urdf = "/home/lmjohann/urdf2casadi/examples/urdf/pantilt.urdf"
root = 'base_link'
tip = "tilt_link"

ok, kdl_tree = kdlurdf.treeFromFile(path_to_urdf)
kdlmodel = kdl_tree.getChain(root,tip)

rbdlmodel = rbdl.loadModel(path_to_urdf)
pbmodel = pb.loadURDF(path_to_urdf, useFixedBase=True, flags = pb.URDF_USE_INERTIA_FROM_FILE)
asd = u2c.URDFparser()
robot_desc = asd.from_file(path_to_urdf)

jlist, names, qmax, qmin = asd.get_joint_info(root, tip)
njoints = asd.get_n_joints(root, tip)

gravity_kdl = kdl.Vector()
gravity_kdl[2] = -9.81
gravity_u2c = [0, 0, -9.81]

#rbdl declarations
q_np = np.zeros(njoints)
qdot_np = np.zeros(njoints)
qddot_np = np.zeros(njoints)
g_rbdl = np.zeros(njoints)
c_rbdl = np.zeros(njoints)
ID_rbdl = np.zeros(njoints)

#kdl declatations
q_kdl = kdl.JntArray(njoints)
qdot_kdl = kdl.JntArray(njoints)
g_kdl = kdl.JntArray(njoints)

#u2c and pybullet declarations
q_none = [None]*njoints
qdot_none = [None]*njoints
qddot_none = [None]*njoints
g_pb = [None]*njoints

qddot = []
for i in range(njoints):
    qddot.append(0.)

qdot = []
for i in range(njoints):
    qdot.append(0.)




The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
/home/lmjohann/urdf2casadi/examples/urdf/pantilt.urdf


In [4]:
g_u2c = asd.get_gravity_rnea(root, tip, gravity_u2c)
timeit_g_u2c = repeat("g_u2c_func()", setup = "from __main__ import g_u2c_func", repeat = nrepeat, number = nitr)
mediantime_g_u2c  = median(timeit_g_u2c)/nitr*1000000

timeit_g_pb = repeat("g_pb_func()", setup = "from __main__ import g_pb_func", repeat = nrepeat, number = nitr)
mediantime_g_pb = median(timeit_g_pb)/nitr*1000000

timeit_g_rbdl = repeat("g_rbdl_func()", setup = "from __main__ import g_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_g_rbdl = median(timeit_g_rbdl)/nitr*1000000

timeit_g_kdl = repeat("g_kdl_func()", setup = "from __main__ import g_kdl_func", repeat = nrepeat, number = nitr)
mediantime_g_kdl = median(timeit_g_kdl)/nitr*1000000


c_u2c = asd.get_coriolis_rnea(root, tip)
timeit_c_u2c = repeat("c_u2c_func()", setup = "from __main__ import c_u2c_func", repeat = nrepeat, number = nitr)
mediantime_c_u2c  = median(timeit_c_u2c)/nitr*1000000

timeit_c_pb = repeat("c_pb_func()", setup = "from __main__ import c_pb_func", repeat = nrepeat, number = nitr)
mediantime_c_pb = median(timeit_c_pb)/nitr*1000000

timeit_c_rbdl = repeat("c_rbdl_func()", setup = "from __main__ import c_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_c_rbdl = median(timeit_c_rbdl)/nitr*1000000

timeit_c_kdl = repeat("c_kdl_func()", setup = "from __main__ import c_kdl_func", repeat = nrepeat, number = nitr)
mediantime_c_kdl = median(timeit_c_kdl)/nitr*1000000

ID_u2c = asd.get_inverse_dynamics_rnea(root, tip, gravity_u2c)
timeit_id_u2c = repeat("ID_u2c_func()", setup = "from __main__ import ID_u2c_func", repeat = nrepeat, number = nitr)
mediantime_id_u2c  = median(timeit_id_u2c)/nitr*1000000

timeit_id_pb = repeat("ID_pb_func()", setup = "from __main__ import ID_pb_func", repeat = nrepeat, number = nitr)
mediantime_id_pb = median(timeit_id_pb)/nitr*1000000

timeit_id_rbdl = repeat("ID_rbdl_func()", setup = "from __main__ import ID_rbdl_func", repeat = nrepeat, number = nitr)
mediantime_id_rbdl = median(timeit_id_rbdl)/nitr*1000000

In [5]:
print "\npendulum_g_u2c=", mediantime_g_u2c
print "\npendulum_id_u2c=", mediantime_id_u2c
print "\npendulum_C_u2c=", mediantime_c_u2c
#print "\npendulum_M_u2c=", mediantime_M_u2c
#print "\npendulum_fd_aba u2c=", mediantime_fd_aba_u2c
#print "\npendulum_fd_crba u2c=", mediantime_fd_crba_u2c

print "\npendulum_g_rbdl=", mediantime_g_rbdl
print "\npendulum_id_rbdl=", mediantime_id_rbdl
print "\npendulum_C_rbdl=", mediantime_c_rbdl

print "\npendulum_g_pb=", mediantime_g_pb
print "\npendulum_id_pb=", mediantime_id_pb
print "\npendulum_C_pb=", mediantime_c_pb

print "\npendulum_g_kdl=", mediantime_g_kdl
print "\npendulum_C_kdl=", mediantime_c_kdl


pendulum_g_u2c= 6.56688213348

pendulum_id_u2c= 25.8663892746

pendulum_C_u2c= 16.4796113968

pendulum_g_rbdl= 3.0345916748

pendulum_id_rbdl= 4.82845306396

pendulum_C_rbdl= 3.72755527496

pendulum_g_pb= 2.39253044128

pendulum_id_pb= 4.0009021759

pendulum_C_pb= 3.20291519165

pendulum_g_kdl= 4.80544567108

pendulum_C_kdl= 5.99098205566
